In [ ]:
import pyspark
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, broadcast, desc, collect_list

from pyspark.sql import Window
from pyspark.sql.types import StructType, StructField, TimestampType, LongType, IntegerType, StringType
import os

In [ ]:
spark = (
    SparkSession
    .builder
    .appName('Streaming III')
    .config("spark.jars.packages", "org.apache.spark:spark-sql-kafka-0-10_2.11:2.4.0")
    .getOrCreate()
)

In [ ]:
base_path = os.getcwd()

project_path = ('/').join(base_path.split('/')[0:-3]) 

users_with_tag_output_path = os.path.join(project_path, 'output/users_with_tag')

In [ ]:
users_with_tags = (    
    spark
    .read
    .option('path', users_with_tag_output_path)
    .load()
).cache()

In [ ]:
def get_person_dynamic(message):
    col_exp = when(lit(False), '')
    for name in names:
        col_exp = col_exp.when(message.like('%{}%'.format(name)), name)
    return col_exp.otherwise('other')

In [ ]:
stream_schema = StructType(
    [
        StructField('question_id', LongType()),
        StructField('body', StringType())
        
    ]
)

In [ ]:
dx = ( 
    spark 
    .readStream 
    .format("kafka") 
    .option("kafka.bootstrap.servers", "localhost:9092")
    .option("subscribe", "test")
    .load()
)

x = (
    dx
    .selectExpr("CAST(value AS STRING) AS v")
    .select(from_json(col('v'), stream_schema))
    .withColumn('physicist', get_person_dynamic(col('body')))
    .writeStream
    .format('memory')
    .outputMode('append')
    .queryName('my_stream')
    .start()
)

